In [131]:
from sage.combinat.ranker import rank_from_list

In [221]:
import functools

In [534]:
class Diagrams(Parent):
    def __init__(self, n, k=2):
        self._n = n
        self._k = k
        Parent.__init__(self, category=Sets().Enumerated().Finite())
        
    def van_der_monde(self):
        return self([i]+[0]*(self._k-1) for i in range(self._n))
    an_element = van_der_monde

    def __iter__(self):
        return iter(RecursivelyEnumeratedSet([self.van_der_monde()],
                                             lambda D: [D2.normal()[0]
                                                        for d in range(1, self._n) 
                                                        for D2,c in D.polarizations(d)]
                                            ))

    class Element(ElementWrapper):
        def __init__(self, parent, value):
            value = tuple(tuple(cell) for cell in value)
            assert len(value) == parent._n
            assert all(len(cell) == parent._k for cell in value)
            ElementWrapper.__init__(self, parent, value)
            
        def __iter__(self):
            return self.value.__iter__()

        def __len__(self):
            return self.value.__len__()
        
        def normal(self):
            S = SymmetricGroup(range(len(self)))
            r = rank_from_list(self)
            sign = S([r(cell) for cell in sorted(self)]).sign()
            return [self.parent() (tuple(sorted(self)) ), sign ]

        def polarizations(self, d=1, j=0, l=1):
            D = self.value
            for i in range(len(D)):
                new_cell = list(D[i])
                a = new_cell[j]
                b = new_cell[l]
                if a < d:
                    continue
                new_cell[j] = a - d
                new_cell[l] = b + 1
                new_cell = tuple(new_cell)
                if new_cell in D:
                    continue
                yield self.parent()(D[:i]+(new_cell,)+D[i+1:]),  prod( (a-i) for i in range(d) )
        
class DiagramSpace(CombinatorialFreeModule):
    def __init__(self, n, k=2, K=QQ):
        CombinatorialFreeModule.__init__(self, K, Diagrams(n,k))
        
    def polarization_on_basis(self, D, d=1, j=0, l=1):
        return self.normal(self.sum_of_terms(D.polarizations(d,j,l)))
    
    def polarization(self, x, d=1, j=0, l=1):
        if isinstance(d, (tuple, list, Partition)):
            for i in d:
                x = self.polarization(x, d=i, j=j, l=l)
            return x
        assert d > 0
        return self._apply_module_endomorphism(x, functools.partial(self.polarization_on_basis,
                                              d=d, j=j, l=l))
    
    def normal(self, x):
        return self._apply_module_endomorphism(x, lambda D: self.term(*D.normal()) )

In [535]:
DD = Diagrams(3)

In [536]:
D = DD( ((1,1), (4,0),(3,3)) ); D

((1, 1), (4, 0), (3, 3))

In [515]:
(1,1) in D

True

In [516]:
(3,0) in D

False

In [499]:
(4,0) in D

True

In [500]:
D.normal()

[((1, 1), (3, 3), (4, 0)), -1]

In [501]:
D = DD(((0, 0), (1, 0), (2, 0)))

In [502]:
D.normal()

[((0, 0), (1, 0), (2, 0)), 1]

In [428]:
DD.list()

[((0, 0), (1, 0), (2, 0)),
 ((0, 0), (1, 0), (0, 1)),
 ((0, 0), (0, 1), (2, 0)),
 ((0, 0), (1, 0), (1, 1)),
 ((0, 0), (1, 0), (0, 2)),
 ((0, 0), (0, 1), (1, 1)),
 ((0, 0), (0, 1), (0, 2))]

In [523]:
QDD = DiagramSpace(3)

In [524]:
QDD.an_element()

2*B[((0, 0), (1, 0), (0, 1))] + 3*B[((0, 0), (0, 1), (2, 0))] + 2*B[((0, 0), (1, 0), (2, 0))]

In [525]:
DD.an_element()

((0, 0), (1, 0), (2, 0))

In [526]:
QDD.normal(QDD.an_element())

-2*B[((0, 0), (0, 1), (1, 0))] + 3*B[((0, 0), (0, 1), (2, 0))] + 2*B[((0, 0), (1, 0), (2, 0))]

In [527]:
QDD.polarization_on_basis(DD.an_element())

B[((0, 0), (0, 1), (2, 0))] + 2*B[((0, 0), (1, 0), (1, 1))]

In [433]:
QDD.polarization_on_basis(DD.an_element(), d=2)

2*B[((0, 0), (1, 0), (0, 1))]

In [434]:
QDD.polarization_on_basis(DD.an_element(), d=3)

0

In [435]:
QDD.polarization(QDD.an_element(), [1,1,1])

12*B[((0, 0), (0, 1), (0, 2))]

In [436]:
QDD.polarization(QDD.an_element(), [2,1])

0

In [437]:
QDD = DiagramSpace(4)

In [438]:
QDD.polarization(QDD.an_element(), [1,1,1,1,1,1])

1440*B[((0, 0), (0, 1), (0, 2), (0, 3))]

In [439]:
QDD.polarization(QDD.an_element(), 3)

12*B[((0, 0), (1, 0), (2, 0), (0, 1))]

In [440]:
QDD.polarization(QDD.an_element(), [3,1])

24*B[((0, 0), (1, 0), (1, 1), (0, 1))]

In [441]:
QDD.polarization(QDD.an_element(), )

6*B[((0, 0), (1, 0), (0, 1), (2, 1))] + 6*B[((0, 0), (0, 1), (1, 1), (3, 0))] + 2*B[((0, 0), (0, 1), (2, 0), (3, 0))] + 4*B[((0, 0), (1, 0), (1, 1), (3, 0))] + 9*B[((0, 0), (0, 1), (2, 0), (2, 1))] + 6*B[((0, 0), (1, 0), (2, 0), (2, 1))]

In [541]:
n = 5
j = 2
i = 2

In [542]:
QDD = DiagramSpace(n)
Delta = QDD.an_element()

In [539]:
def the_dim(i, j):
    P = Partitions(binomial(n,2)-i, length=j)   
    return len(QDD.echelon_form([QDD.polarization(Delta, p) for p in P]))

In [544]:
q, t = QQ['q,t'].gens()
sum( q^i * t^j * the_dim(i,j) 
    for i in range(binomial(n,2)+1)
    for j in range(binomial(n,2)+1))\
- qt_catalan_number(n)

q^5*t^2 + q^4*t^3

In [545]:
i = 5
j = 2
P = Partitions(binomial(n,2)-i, length=j)   
[QDD.polarization(Delta, p) for p in P]

[-144*B[((0, 0), (0, 1), (1, 0), (2, 0), (2, 1))] + 72*B[((0, 0), (0, 1), (0, 2), (1, 0), (2, 0))] - 96*B[((0, 0), (0, 1), (1, 0), (1, 1), (3, 0))],
 144*B[((0, 0), (0, 1), (1, 0), (2, 0), (2, 1))] + 96*B[((0, 0), (0, 1), (1, 0), (1, 1), (3, 0))]]

In [546]:
qt_catalan_number(5)

q^10 + q^9*t + q^8*t^2 + q^7*t^3 + q^6*t^4 + q^5*t^5 + q^4*t^6 + q^3*t^7 + q^2*t^8 + q*t^9 + t^10 + q^8*t + q^7*t^2 + q^6*t^3 + q^5*t^4 + q^4*t^5 + q^3*t^6 + q^2*t^7 + q*t^8 + q^7*t + 2*q^6*t^2 + 2*q^5*t^3 + 2*q^4*t^4 + 2*q^3*t^5 + 2*q^2*t^6 + q*t^7 + q^6*t + q^5*t^2 + 2*q^4*t^3 + 2*q^3*t^4 + q^2*t^5 + q*t^6 + q^4*t^2 + q^3*t^3 + q^2*t^4

In [533]:
i = 3; j = 2
P = Partitions(binomial(n,2)-i, length=j)
[QDD.polarization(Delta, p) for p in P]

[-12*B[((0, 0), (0, 1), (1, 0), (2, 1))] + 12*B[((0, 0), (0, 2), (1, 0), (2, 0))] - 18*B[((0, 0), (0, 1), (0, 2), (2, 0))] - 12*B[((0, 0), (0, 1), (1, 1), (2, 0))] + 12*B[((0, 0), (0, 1), (0, 2), (1, 0))]]

In [454]:
from sage.combinat.q_analogues import *

In [455]:
qt_catalan_number(4)

q^6 + q^5*t + q^4*t^2 + q^3*t^3 + q^2*t^4 + q*t^5 + t^6 + q^4*t + q^3*t^2 + q^2*t^3 + q*t^4 + q^3*t + q^2*t^2 + q*t^3